# Data Exploration and Cleanup

## Overview

This project analyzes anonymized data from Divvy, a rideshare biking service in Chicago, IL. We chose to examine data in the third quarter of 2019 which includes the summer months and high volume of riders compared to other quarters. 

We wanted to uncover interesting patterns in the data. **We asked ourselves:**
* Are there routes and stations that are more popular than others?
* Are there any specific bikes that were rented a surprising amount?
* Are there any relationships between the age and/or gender when comparing one-time customers or yearly subscribers?
* How long, on average, are trips made by Divvy bike?

## Observations

**Popular Stations**
* The most popular station by far is Streeter Dr. and Grand Ave. which is located on Navy Pier. It is a popular destination for both tourists and locals. Indeed, this station appeared as either the starting or ending station on many of the most popular routes. 

**Popular Routes**
* All of the most frequented routes were in the loop and along the lakefront. Without knowing, we suspect that this is again a combination of tourists and commuters making use of the bikes in downtown Chicago. 

**Customers vs. Subscribers:** 
* There are far more subscribers than customers who ride in the Chicago Divvy network, about 84% to 16%
* This data may be skewed. We made an early call to drop any data with null values. We suspect that the dropped cells are more likely to be customers, one-time or infrequent users.

**Gender:**
* There are far more male Divvy riders than female - about 70% to 30%.
* It is fairly the same gender breakdown when it comes to customer (about 70/30) and subscriber (about 60/40).
* Divvy did not collect gender data beyond male and female, which means portions of Chicago's non-binary population are not accounted for in this dataset. 
    
**Age:**
* Riders aged 20-29 made up 47% of overall ridership with people aged 30-39 coming in second with 30% of overall ridership.
* The breakdown per usertype, customer versus subscriber, was similar to the overall categories. 

# Importing & Cleaning Data

# Notes are in-line with code

In [36]:
# Import Dependencies
from matplotlib import pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
import requests
import json
import os
import gmaps

# Import API key
# from config import g_key

In [37]:
# Import data file
divvy_df = pd.read_csv('resources/Divvy_Trips_2019_Q3.csv')

# First, we displayed the dataframe head to quickly identify any obvious items in need of cleaning. 'gender' and 'birthyear' immediately stand out as columns in need of cleaning.

In [38]:
divvy_df.shape

(1640718, 12)

In [39]:
# Display sample of dataframe
divvy_df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,23479388,2019-07-01 00:00:27,2019-07-01 00:20:41,3591,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0
1,23479389,2019-07-01 00:01:16,2019-07-01 00:18:44,5353,"1,048.0",381,Western Ave & Monroe St,203,Western Ave & 21st St,Customer,NaN,NaN
2,23479390,2019-07-01 00:01:48,2019-07-01 00:27:42,6180,"1,554.0",313,Lakeview Ave & Fullerton Pkwy,144,Larrabee St & Webster Ave,Customer,NaN,NaN
3,23479391,2019-07-01 00:02:07,2019-07-01 00:27:10,5540,"1,503.0",313,Lakeview Ave & Fullerton Pkwy,144,Larrabee St & Webster Ave,Customer,NaN,NaN
4,23479392,2019-07-01 00:02:13,2019-07-01 00:22:26,6014,"1,213.0",168,Michigan Ave & 14th St,62,McCormick Place,Customer,NaN,NaN


# Null rows dropped. We noticed that since many customers do not have accounts, insights focused on gender and age will be more limited. Nearly 300,000 rows were dropped, mostly of the customer user type.

In [40]:
# Remove null rows (if needed)
divvy_df.dropna(inplace=True)
divvy_df.shape

(1353368, 12)

In [41]:
# Find column names
divvy_df.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

# Next, to make it easier to analyze the users by age, we converted 'birthyear' to the user's age in 2019.

In [42]:
# Convert birthyear to age
divvy_df['age'] = (2019 - divvy_df['birthyear'])
divvy_df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,age
0,23479388,2019-07-01 00:00:27,2019-07-01 00:20:41,3591,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0,27.0
5,23479393,2019-07-01 00:02:21,2019-07-01 00:07:31,4941,310.0,300,Broadway & Barry Ave,232,Pine Grove Ave & Waveland Ave,Subscriber,Male,1990.0,29.0
18,23479406,2019-07-01 00:06:51,2019-07-01 00:26:22,2758,"1,171.0",624,Dearborn St & Van Buren St,237,MLK Jr Dr & 29th St,Subscriber,Male,1995.0,24.0
20,23479408,2019-07-01 00:08:24,2019-07-01 00:20:07,2447,703.0,239,Western Ave & Leland Ave,455,Maplewood Ave & Peterson Ave,Subscriber,Male,1970.0,49.0
21,23479409,2019-07-01 00:08:40,2019-07-01 00:27:50,6336,"1,150.0",257,Lincoln Ave & Waveland Ave,295,Broadway & Argyle St,Subscriber,Male,1993.0,26.0


# We added bins to help with the graphic representation of our data.

In [43]:
# Create bins for age
bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Create the names for the bins
group_names = ["11-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89"]
divvy_df["Age Group"] = pd.cut(divvy_df["age"], bins, labels=group_names, include_lowest=True)
divvy_df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,age,Age Group
0,23479388,2019-07-01 00:00:27,2019-07-01 00:20:41,3591,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0,27.0,20-29
5,23479393,2019-07-01 00:02:21,2019-07-01 00:07:31,4941,310.0,300,Broadway & Barry Ave,232,Pine Grove Ave & Waveland Ave,Subscriber,Male,1990.0,29.0,20-29
18,23479406,2019-07-01 00:06:51,2019-07-01 00:26:22,2758,"1,171.0",624,Dearborn St & Van Buren St,237,MLK Jr Dr & 29th St,Subscriber,Male,1995.0,24.0,20-29
20,23479408,2019-07-01 00:08:24,2019-07-01 00:20:07,2447,703.0,239,Western Ave & Leland Ave,455,Maplewood Ave & Peterson Ave,Subscriber,Male,1970.0,49.0,40-49
21,23479409,2019-07-01 00:08:40,2019-07-01 00:27:50,6336,"1,150.0",257,Lincoln Ave & Waveland Ave,295,Broadway & Argyle St,Subscriber,Male,1993.0,26.0,20-29


# The 'tripduration' column is in seconds, but it did not seem to be accurate upon further analysis. Here we manually created the time in minutes as the 'time difference' column to confirm the accuracy of the time. 

In [44]:
divvy_df['time difference'] = ((pd.to_datetime(divvy_df['end_time']) - 
                            pd.to_datetime(divvy_df['start_time']))
                                .dt.total_seconds() / 60)
divvy_df

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,age,Age Group,time difference
0,23479388,2019-07-01 00:00:27,2019-07-01 00:20:41,3591,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0,27.0,20-29,20.233333
5,23479393,2019-07-01 00:02:21,2019-07-01 00:07:31,4941,310.0,300,Broadway & Barry Ave,232,Pine Grove Ave & Waveland Ave,Subscriber,Male,1990.0,29.0,20-29,5.166667
18,23479406,2019-07-01 00:06:51,2019-07-01 00:26:22,2758,"1,171.0",624,Dearborn St & Van Buren St,237,MLK Jr Dr & 29th St,Subscriber,Male,1995.0,24.0,20-29,19.516667
20,23479408,2019-07-01 00:08:24,2019-07-01 00:20:07,2447,703.0,239,Western Ave & Leland Ave,455,Maplewood Ave & Peterson Ave,Subscriber,Male,1970.0,49.0,40-49,11.716667
21,23479409,2019-07-01 00:08:40,2019-07-01 00:27:50,6336,"1,150.0",257,Lincoln Ave & Waveland Ave,295,Broadway & Argyle St,Subscriber,Male,1993.0,26.0,20-29,19.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640713,25223635,2019-09-30 23:56:27,2019-10-01 00:06:20,5690,593.0,84,Milwaukee Ave & Grand Ave,134,Peoria St & Jackson Blvd,Subscriber,Male,1993.0,26.0,20-29,9.883333
1640714,25223636,2019-09-30 23:57:02,2019-10-01 00:02:13,5079,310.0,268,Lake Shore Dr & North Blvd,85,Michigan Ave & Oak St,Subscriber,Male,1992.0,27.0,20-29,5.183333
1640715,25223637,2019-09-30 23:57:45,2019-10-01 00:01:41,5535,235.0,423,University Ave & 57th St,426,Ellis Ave & 60th St,Customer,Male,2000.0,19.0,11-19,3.933333
1640716,25223638,2019-09-30 23:57:47,2019-10-01 00:04:41,6120,414.0,110,Dearborn St & Erie St,31,Franklin St & Chicago Ave (Temp),Subscriber,Male,1986.0,33.0,30-39,6.900000


# Here we converted the 'start_time' and 'end_time' to the datetime format 

In [45]:
# Convert date columns to datetime format
divvy_df['start_time'] = pd.to_datetime(divvy_df['start_time'])
divvy_df['end_time'] = pd.to_datetime(divvy_df['end_time'])

# Then with the columns in the datetime format, we sliced the date from the time into their own columns. 

In [46]:
# Split date from time and create columns
divvy_df['start_date'] = divvy_df['start_time'].dt.date
divvy_df['start_time'] = divvy_df['start_time'].dt.time
divvy_df['end_date'] = divvy_df['end_time'].dt.date
divvy_df['end_time'] = divvy_df['end_time'].dt.time
divvy_df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,age,Age Group,time difference,start_date,end_date
0,23479388,00:00:27,00:20:41,3591,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0,27.0,20-29,20.233333,2019-07-01,2019-07-01
5,23479393,00:02:21,00:07:31,4941,310.0,300,Broadway & Barry Ave,232,Pine Grove Ave & Waveland Ave,Subscriber,Male,1990.0,29.0,20-29,5.166667,2019-07-01,2019-07-01
18,23479406,00:06:51,00:26:22,2758,"1,171.0",624,Dearborn St & Van Buren St,237,MLK Jr Dr & 29th St,Subscriber,Male,1995.0,24.0,20-29,19.516667,2019-07-01,2019-07-01
20,23479408,00:08:24,00:20:07,2447,703.0,239,Western Ave & Leland Ave,455,Maplewood Ave & Peterson Ave,Subscriber,Male,1970.0,49.0,40-49,11.716667,2019-07-01,2019-07-01
21,23479409,00:08:40,00:27:50,6336,"1,150.0",257,Lincoln Ave & Waveland Ave,295,Broadway & Argyle St,Subscriber,Male,1993.0,26.0,20-29,19.166667,2019-07-01,2019-07-01


# To make the dataframe easier to read, in light of the new columns, we reorganized the date and time columns to be next to each other.

In [47]:
# Organize columns in dataframe
divvy_df = divvy_df[["trip_id", "bikeid", "start_date", "start_time", "time difference", "end_date", "end_time", "tripduration", "from_station_id", "from_station_name", "to_station_id", "to_station_name", "usertype", "gender", "birthyear", "age", "Age Group"]]
divvy_df.head(10)

,trip_id,bikeid,start_date,start_time,time difference,end_date,end_time,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,age,Age Group
0,23479388,3591,2019-07-01,00:00:27,20.233333,2019-07-01,00:20:41,"1,214.0",117,Wilton Ave & Belmont Ave,497,Kimball Ave & Belmont Ave,Subscriber,Male,1992.0,27.0,20-29
5,23479393,4941,2019-07-01,00:02:21,5.166667,2019-07-01,00:07:31,310.0,300,Broadway & Barry Ave,232,Pine Grove Ave & Waveland Ave,Subscriber,Male,1990.0,29.0,20-29
18,23479406,2758,2019-07-01,00:06:51,19.516667,2019-07-01,00:26:22,"1,171.0",624,Dearborn St & Van Buren St,237,MLK Jr Dr & 29th St,Subscriber,Male,1995.0,24.0,20-29
20,23479408,2447,2019-07-01,00:08:24,11.716667,2019-07-01,00:20:07,703.0,239,Western Ave & Leland Ave,455,Maplewood Ave & Peterson Ave,Subscriber,Male,1970.0,49.0,40-49
21,23479409,6336,2019-07-01,00:08:40,19.166667,2019-07-01,00:27:50,"1,150.0",257,Lincoln Ave & Waveland Ave,295,Broadway & Argyle St,Subscriber,Male,1993.0,26.0,20-29
29,23479418,4525,2019-07-01,00:15:06,21.566667,2019-07-01,00:36:40,"1,294.0",240,Sheridan Rd & Irving Park Rd,326,Clark St & Leland Ave,Customer,Male,1987.0,32.0,30-39
30,23479419,1132,2019-07-01,00:18:53,27.066667,2019-07-01,00:45:57,"1,624.0",6,Dusable Harbor,321,Wabash Ave & 9th St,Customer,Female,1998.0,21.0,20-29
31,23479420,5166,2019-07-01,00:18:53,27.016667,2019-07-01,00:45:54,"1,621.0",6,Dusable Harbor,321,Wabash Ave & 9th St,Customer,Female,1997.0,22.0,20-29
32,23479421,240,2019-07-01,00:19:41,26.216667,2019-07-01,00:45:54,"1,573.0",6,Dusable Harbor,321,Wabash Ave & 9th St,Customer,Female,1997.0,22.0,20-29
34,23479423,1830,2019-07-01,00:22:56,16.716667,2019-07-01,00:39:39,"1,003.0",203,Western Ave & 21st St,203,Western Ave & 21st St,Subscriber,Male,1970.0,49.0,40-49
